In [ ]:
# 把ipynb的运行根目录放在../src下
import os
import sys
sys.path.append(os.path.abspath('..'))
from BMGs import BMGs
import pandas as pd
from config import *

### 插入s到new_BMGs中

In [ ]:
data = pd.read_excel('../designs/ddpg_1500/new_BMGs.xlsx')
for i in range(0, len(data)):
    BMG = BMGs(bmg_s=data.iloc[i]['BMGs'])
    s = BMG.s
    for j in range(0, len(CompositionColumns)):
        data.loc[i, CompositionColumns[j]] = s[j]
data.to_excel('../designs/ddpg_1500/new_BMGs.xlsx', index=False)

### 统计元素出现次数

In [ ]:
from pyecharts.charts import Pie
from pyecharts import options as opts
from pyecharts.globals import ThemeType

def draw_pie_chart(data, title):
    # Convert to the format required by pyecharts
    data_pair = [(k, v) for k, v in data.items()]
    print(data_pair)
    
    # Create a pie chart with pyecharts
    pie = (
        Pie(init_opts=opts.InitOpts(theme=ThemeType.MACARONS))
        .add(
            "",
            data_pair,
            radius=["30%", "70%"], 
            center=["60%", "50%"],  # Adjust center to move the pie chart
            rosetype="radius",  # Use rosetype to create a rose chart
        )
        .set_global_opts(
            # title_opts=opts.TitleOpts(title=title, title_textstyle_opts=opts.TextStyleOpts(font_weight="bold")),
            legend_opts=opts.LegendOpts(
                orient="vertical", 
                pos_top="15%", 
                pos_left="2%",
                textstyle_opts=opts.TextStyleOpts(font_weight="bold")  # Set legend text to bold
            )
        )
        .set_series_opts(
            label_opts=opts.LabelOpts(
                formatter="{b}: {d}%",
                position="outside",
                font_size=10,
                is_show=True,
                color="#000",
                distance=20,  # Adjust the distance from label to pie chart
                font_weight="bold"  # Set label text to bold
            )
        )
    )
    pie.render(f'{title}.html')


In [ ]:
def get_base_matrix_count(data_path):
    data = pd.read_excel(data_path)
    base_matrix_count = {}
    for i in range(0, len(data)):
        bmg = BMGs(bmg_s=data.iloc[i]['BMGs'])
        base_matrix = bmg.get_base_matrix()
        if base_matrix in base_matrix_count:
            base_matrix_count[base_matrix] += 1
        else:
            base_matrix_count[base_matrix] = 1
    base_matrix_count = dict(sorted(base_matrix_count.items(), key=lambda x: x[1], reverse=True))
    return base_matrix_count

In [ ]:
original_base_matrix_count = get_base_matrix_count('../data/ALL_data_grouped_processed.xlsx')
new_bmgs_base_matrix_count = get_base_matrix_count('../ckpts/ddpg/new_BMGs.xlsx')
draw_pie_chart(original_base_matrix_count, 'Original BMGs Base Matrix Distribution')
print('Original base matrix count:', len(original_base_matrix_count.keys()))
draw_pie_chart(new_bmgs_base_matrix_count, 'New BMGs Base Matrix Distribution in Training.')
print('New BMGs base matrix count:', len(new_bmgs_base_matrix_count.keys()))

In [ ]:
def get_optional_matrix_count(data_path, optional_elements):
    # Load the data from the Excel file
    data = pd.read_excel(data_path)

    # Initialize a statistics to store the counts
    optional_matrix_count = {}

    # Iterate over each element in the list of optional elements
    for element in optional_elements:
        if element in data.columns:  # Check if the column exists in the DataFrame
            # Count the non-zero values in the column
            count = (data[element] != 0).sum()
            optional_matrix_count[element] = count
        else:
            # If the column does not exist, set the count to None or 0
            optional_matrix_count[element] = None  # or use 0 as a default value

    # Return the statistics with the count of non-zero values
    return optional_matrix_count

In [ ]:
optional_matrix_count = get_optional_matrix_count('../designs/ddpg_1500/new_BMGs.xlsx', list(ExploreBases[0][1].keys()))
draw_pie_chart(optional_matrix_count, 'Zr Base BMGs Optional Matrix Distribution in Testing.')

### 目标性能绘制

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_scatter(df1: pd.DataFrame, df2: pd.DataFrame, x_col: str, y_col: str, title: str):

    # Drop rows where x_col or y_col is None
    df1 = df1.dropna(subset=[x_col, y_col])
    df2 = df2.dropna(subset=[x_col, y_col])
    print(len(df1), len(df2))
    
    # Set the style of the plot
    sns.set(style="whitegrid")
    
    # Create a scatter plot
    plt.figure(figsize=(12, 8))
    plt.scatter(df1[x_col], df1[y_col], color='blue', label='Existing BMGs', alpha=0.6, edgecolor='w', s=100)
    plt.scatter(df2[x_col], df2[y_col], color='red', label='Newly Designed BMGs', alpha=0.6, edgecolor='w', s=100)
    
    # Add title and labels
    plt.title(title, fontsize=16, fontweight='bold')
    plt.xlabel(x_col, fontsize=14, fontweight='bold')
    plt.ylabel(y_col, fontsize=14, fontweight='bold')
    plt.legend(fontsize=12)
    
    # Add a grid and adjust the layout
    plt.grid(False)
    plt.tight_layout()
    
    # Show the plot
    plt.savefig(f'{title}.png', dpi=500)
    plt.close()


In [ ]:
df1 = pd.read_excel('/data/home/yeyongyu/SHU/ReinforceMatDesign/data/ALL_data_grouped_processed.xlsx')
df2 = pd.read_excel('/data/home/yeyongyu/SHU/ReinforceMatDesign/ckpts/ddpg/new_BMGs.xlsx')
plot_scatter(df1, df2, 'yield(MPa)', 'Ε(%)', 'Yield(MPa) vs. Ε(%) Scatter Plot')

In [ ]:
df1 = pd.read_excel('/data/home/yeyongyu/SHU/ReinforceMatDesign/data/ALL_data_grouped_processed.xlsx')
df3 = pd.read_excel('/data/home/yeyongyu/SHU/ReinforceMatDesign/designs/ddpg_1500/new_BMGs.xlsx')
df1 = df1[df1['Ag'] != 0]
df3 = df3[df3['Ag'] != 0]
plot_scatter(df1, df3, 'Ag', 'yield(MPa)', 'Yield(MPa) Change with Ag Composition')
plot_scatter(df1, df3, 'Ag', 'Ε(%)', 'E(%) Change with Ag Composition')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_scatter_colorbar(df: pd.DataFrame, x_col: str, y_col: str, color_bar_col: str, title: str, colormap: str = 'plasma_r'):
    """
    绘制散点图，并使用 color_bar_col 的值绘制色条。
    
    参数:
    df: pandas.DataFrame - 数据框
    x_col: str - x轴数据列名
    y_col: str - y轴数据列名
    color_bar_col: str - 用于色条的数据列名
    title: str - 图表标题
    colormap: str - 色条的颜色映射
    """
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(df[x_col], df[y_col], c=df[color_bar_col], cmap=colormap)
    colorbar = plt.colorbar(scatter)
    colorbar.set_label(color_bar_col)
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title(title)
    plt.grid(False)
    plt.savefig(f'{title}.png', dpi=500)
    plt.close()


In [ ]:
df = pd.read_excel('/data/home/yeyongyu/SHU/ReinforceMatDesign/designs/ddpg_1500/new_BMGs.xlsx')
df = df[df['Ag'] != 0]
plot_scatter_colorbar(df, 'yield(MPa)', 'Ε(%)', 'Ag', 'Yield(MPa) vs. Ε(%) Scatter Plot with Ag Composition')

In [ ]:
# 计算df中Ag 0-5， 5-10, 10-15的yield(MPa)和Ε(%)的均值和标准差，还有数量
def get_stats(df: pd.DataFrame, element: str, x_col: str, y_col: str):
    stats = {}
    for i in range(0, 5):
        lower_bound = i * 3
        upper_bound = (i + 1) * 3
        key = f'{lower_bound}-{upper_bound}'
        subset = df[(df[element] >= lower_bound) & (df[element] < upper_bound)]
        stats[key] = {
            'count': len(subset),
            'mean_yield': subset[x_col].mean(),
            'std_yield': subset[x_col].std(),
            'mean_Ε': subset[y_col].mean(),
            'std_Ε': subset[y_col].std()
        }
    return stats

stats = get_stats(df, 'Ag', 'yield(MPa)', 'Ε(%)')
print(stats)

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd

def plot_element_influence(statistics, name):
    """
    根据提供的字典数据绘制图表。
    
    参数:
    statistics: dict - 输入的数据字典
    """
    # 将字典转换为DataFrame
    data = {
        'Range': list(statistics.keys()),
        'Count': [statistics[key]['count'] for key in statistics.keys()],
        'Mean Yield': [statistics[key]['mean_yield'] for key in statistics.keys()],
        'Std Yield': [statistics[key]['std_yield'] for key in statistics.keys()],
        'Mean Ε': [statistics[key]['mean_Ε'] for key in statistics.keys()],
        'Std Ε': [statistics[key]['std_Ε'] for key in statistics.keys()]
    }

    df = pd.DataFrame(data)

    # 创建图表
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # 绘制mean_yield及其标准差
    bar_width = 0.4
    bars = ax1.bar(df['Range'], df['Mean Yield'], yerr=df['Std Yield'], label='Mean Yield', color='#1f77b4', capsize=5, width=bar_width, alpha=0.7, edgecolor='grey')

    # 设置Y轴标签
    ax1.set_xlabel('Range')
    ax1.set_ylabel('Mean Yield', color='#1f77b4')
    ax1.tick_params(axis='y', labelcolor='#1f77b4')

    # 创建第二个Y轴并绘制mean_Ε及其标准差
    ax2 = ax1.twinx()
    ax2.plot(df['Range'], df['Mean Ε'], label='Mean Ε', color='#ff7f0e', marker='o', linestyle='-', linewidth=2, markersize=6)
    ax2.errorbar(df['Range'], df['Mean Ε'], yerr=df['Std Ε'], fmt='o', color='#ff7f0e', capsize=5)

    # 设置第二个Y轴标签
    ax2.set_ylabel('Mean Ε', color='#ff7f0e')
    ax2.tick_params(axis='y', labelcolor='#ff7f0e')

    # 添加标题
    plt.title('Mean Yield and Mean Ε with Standard Deviations')
    plt.grid(False)

    # 调整图例
    fig.legend(loc='upper left', bbox_to_anchor=(0.1,0.9))

    # 显示图表
    plt.savefig(f"{name}.png", dpi=500)
    plt.close()

In [ ]:
plot_element_influence(stats, 'Mean Yield and Mean Ε with Standard Deviations.png')